In [1]:
import tensorflow as tf
from convert_to_record import read_and_decode_image

def train_data(images_tfrecord, masks_tfrecord, size=(580, 420)):
    images_queue = tf.train.string_input_producer([images_tfrecord])
    image = read_and_decode_image(images_queue, size)
    
    masks_queue = tf.train.string_input_producer([masks_tfrecord])
    mask = tf.to_float(tf.greater(read_and_decode_image(masks_queue, size)[:,:, 0], 0))
    return image, mask

def preprocess(image):
    # First, the image is essentially grayscale.
    gray_image = tf.image.rgb_to_grayscale(image)
    
    # Then we want to normalize.
    normalized_image = (1.0/255)*(tf.to_float(gray_image) - 127.0)
    
    # We'll likely also want to produce random perturbation of the
    # data to synthetically increase the size of the data set
    # TODO(kjchavez): Add random jitter.
    
    return normalized_image

def batched_train_data(images_tfrecord, masks_tfrecord, batch_size=32, size=(580, 420)):
    image, mask = train_data(images_tfrecord, masks_tfrecord, size=size)
    image = preprocess(image)
    image_batch, mask_batch = tf.train.batch([image, mask], batch_size)
    return image_batch, mask_batch

In [2]:
# The predictive model.

def predict(X):
    """ Produces logits for each pixel in a training image, corresponding to
        the probability that it is part of the sought-after nerve.
    """
    # Allocate variables on GPU.
    with tf.device("/gpu:0"):
        W1 = tf.get_variable("w1", [5, 5, 1, 32], initializer=tf.truncated_normal_initializer(stddev=0.2))
        b1 = tf.get_variable("b1", [32], initializer=tf.constant_initializer(0.0))
        W2 = tf.get_variable("w2", [1, 1, 32, 1], initializer=tf.truncated_normal_initializer(stddev=0.2))
        b2 = tf.get_variable("b2", [1], initializer=tf.constant_initializer(0.0))
        
    conv1 = tf.nn.conv2d(X, W1, strides=(1,1,1,1), padding='SAME', name="conv")
    a1 = tf.nn.relu(tf.nn.bias_add(conv1, b1))
    conv2 = tf.nn.conv2d(a1, W2, strides=(1,1,1,1), padding='SAME', name="net_in_net")
    return tf.squeeze(tf.nn.bias_add(conv2, b2))

def prob(logits):
    output_prob = tf.nn.sigmoid(logits)
    return output_prob

def xentropy_loss(logits, y):
    xentropy = tf.nn.sigmoid_cross_entropy_with_logits(logits, y, name="xentropy_loss")
    return tf.reduce_mean(xentropy)

def predict_mask(probs, threshold=0.5):
    bool_mask = probs > threshold
    return bool_mask

def dice_coefficient(pred, label):
    pred = tf.to_int32(pred)
    label = tf.to_int32(label)
    positive_in_pred = tf.reduce_sum(pred, reduction_indices=[1, 2])
    positive_in_label = tf.reduce_sum(label, reduction_indices=[1, 2])
    denom = tf.to_float(positive_in_pred + positive_in_label)
    numer = 2.0 * tf.to_float(tf.reduce_sum(pred * label, reduction_indices=[1,2]))

    return numer / denom

In [4]:
from train import *

# Build a model
X, y = batched_train_data("train_images.tfrecord", "train_masks.tfrecord")

with tf.variable_scope("model", reuse=False) as scope:
    logits = predict(X)
    y_pred = prob(logits)

loss = xentropy_loss(logits, y)
score = dice_coefficient(predict_mask(y_pred, threshold=0.5), y)
avg_score = tf.reduce_mean(score)

lr = 1e-3
train_op = get_sgd_train_op(loss, lr)

def evaluate():
    sess = tf.get_default_session()
    avg_score_value = sess.run(avg_score)
    return "Average Dice Coefficient on training minibatch: %0.4f" % avg_score_value

In [ ]:
train(train_op, evaluate, loss, 1000,
      print_every_n=10,
      eval_every_n=100,
      snapshot_every_n=100,
      summarize_every_n=10,
      diverge_threshold=1.0)

Loss @ iteration 00000: 0.777571
Loss @ iteration 00010: 0.777847
Loss @ iteration 00020: 0.777602
Loss @ iteration 00030: 0.762776
Loss @ iteration 00040: 0.763489
Loss @ iteration 00050: 0.758508
Loss @ iteration 00060: 0.748659
Loss @ iteration 00070: 0.748529
Loss @ iteration 00080: 0.731098
Loss @ iteration 00090: 0.729687
Loss @ iteration 00100: 0.715489
Average Dice Coefficient on training minibatch: 0.0125
Loss @ iteration 00110: 0.714883
Loss @ iteration 00120: 0.705453
Loss @ iteration 00130: 0.702829
Loss @ iteration 00140: 0.699686
Loss @ iteration 00150: 0.692023
Loss @ iteration 00160: 0.681758
Loss @ iteration 00170: 0.684437
Loss @ iteration 00180: 0.678026
Loss @ iteration 00190: 0.669673
Loss @ iteration 00200: 0.668086
Average Dice Coefficient on training minibatch: 0.0089
Loss @ iteration 00210: 0.658942
Loss @ iteration 00220: 0.656159
Loss @ iteration 00230: 0.647740
Loss @ iteration 00240: 0.643756
Loss @ iteration 00250: 0.636938
Loss @ iteration 00260: 0.631776

In [ ]:
# Run the model with raw TensorFlow, no helper function.
"""
import numpy as np
num_batches = 10

init_op = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init_op)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    # Do stuff
    print "Starting training."
    for n in xrange(num_batches):
        _, y_val, y_pred_val, loss_value, avg_score_value = sess.run([train_op, y, y_pred, loss, score])
        print loss_value
        print avg_score_value
    
    coord.request_stop()
    coord.join(threads)
"""

In [8]:
%matplotlib inline
import matplotlib.pyplot as plt

sess = tf.get_default_session()
y_val, y_pred_val = sess.run([y_val, y_pred_val])

idx = 1
#t = np.reshape(y_val[idx], (580, 420))
#t_hat = np.reshape(y_pred_val[idx], (580, 420)) > 0.5
t = y_val[idx]
t_hat = y_pred_val[idx] > 0.5

plt.subplot(121)
plt.imshow(t, cmap='gray')
plt.subplot(122)
plt.imshow(t_hat.astype(np.float), cmap='gray', clim=(0, 1.0))

AttributeError: 'NoneType' object has no attribute 'run'